In [62]:
import pandas as pd
import numpy as np

df = pd.read_excel("dcmotor_simdata.xls")
t = np.array(df["Var1"].array)

KePhi = 1.6
TL = 20

W = np.array(df["Var2"].array)
dW = np.concatenate((np.zeros(1), np.diff(W)/np.diff(t)))

Va = np.array(df["Var4"].array)

Ia = np.array(df["Var5"].array)
dIa = np.concatenate((np.zeros(1), np.diff(Ia)/np.diff(t)))

df = pd.DataFrame({"t": t, "W": W, "dw": dW, "Va": Va, "Ia": Ia, "dIa": dIa})

In [63]:
Xy = np.zeros((4,1))
XX_T = np.zeros((4,4))

for k in range(len(t)):
    y = np.vstack([Va[k] - KePhi*W[k], KePhi*Ia[k] - TL])
    X = np.array([
        [Ia[k], dIa[k], 0, 0],
        [0, 0, dW[k], W[k]]
    ])
    Xy += X.T @ y
    XX_T += X.T @ X 

theta = np.linalg.inv(XX_T) @ Xy
print(theta)

# X = np.vstack([
#     [Ia, dIa, np.zeros(len(t)), np.zeros(len(t))], 
#     [np.zeros(len(t)), np.zeros(len(t)), dW, W]
# ])
# y = np.vstack([Va - KePhi*W, KePhi*Ia - TL])

# theta = np.linalg.pinv(X.T).dot(y)
# print(theta)


[[ 0.49882114]
 [ 0.082639  ]
 [ 4.98082508]
 [-0.18575007]]
